# spell checking


do old-new tests informally here.

general approach:
- run block of code to save off results for later comparison
- new block of same code that gets run once changes made
- compare results


In [1]:
import stitches as stitches

In [2]:
import pandas as pd
import pkg_resources
import xarray as xr
import numpy as np

# For help with plotting
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6

In [4]:
# read in the package data of all ESMs-Scenarios-ensemble members avail.
path = pkg_resources.resource_filename('stitches', 'data/matching_archive.csv')
data = pd.read_csv(path)

# subset to the target
target_data = data[data["model"] == "CanESM5"].copy()
target_data = target_data[target_data["experiment"] == 'ssp245']
target_data = target_data[target_data["ensemble"].isin(['r1i1p1f1'])]
target_data = target_data.reset_index(drop=True)
print(target_data.head())


   ensemble variable    model experiment  start_yr  end_yr  year        fx  \
0  r1i1p1f1      tas  CanESM5     ssp245      1850    1858  1854 -1.289870   
1  r1i1p1f1      tas  CanESM5     ssp245      1859    1867  1863 -1.268776   
2  r1i1p1f1      tas  CanESM5     ssp245      1868    1876  1872 -1.215083   
3  r1i1p1f1      tas  CanESM5     ssp245      1877    1885  1881 -1.195105   
4  r1i1p1f1      tas  CanESM5     ssp245      1886    1894  1890 -1.250430   

         dx  
0 -0.003368  
1  0.009320  
2  0.004758  
3 -0.014627  
4  0.014695  


In [5]:
archive_data = data[data["model"] == "CanESM5"]
archive_data = archive_data[archive_data["experiment"].isin([
    'ssp126',  'ssp370', 'ssp585'])]
print(archive_data.head())

       ensemble variable    model experiment  start_yr  end_yr  year  \
6996  r10i1p1f1      tas  CanESM5     ssp126      1850    1858  1854   
6997  r10i1p1f1      tas  CanESM5     ssp126      1859    1867  1863   
6998  r10i1p1f1      tas  CanESM5     ssp126      1868    1876  1872   
6999  r10i1p1f1      tas  CanESM5     ssp126      1877    1885  1881   
7000  r10i1p1f1      tas  CanESM5     ssp126      1886    1894  1890   

            fx        dx  
6996 -1.360399  0.016472  
6997 -1.297474 -0.001508  
6998 -1.200856  0.010901  
6999 -1.218503 -0.016847  
7000 -1.285527  0.006000  


# Pre fix  block
- save off recipes
- MUST NOT re-run this block once changes have been enacted

In [5]:
my_recipes = stitches.make_recipe(target_data, archive_data,
                                  tol = 0.06, N_matches = 4, reproducible=True)

print(my_recipes.head())
# my_recipes.to_csv('old_recipes.csv', index=False)
# stitched_global_temp = stitches.gmat_stitching(my_recipes)
# stitched_global_temp.to_csv('old_tgavs.csv', index=False)

You have requested more recipes than possible for at least one target trajectories, returning what can
  target_start_yr target_end_yr archive_experiment archive_variable  \
0            1850          1858         historical              tas   
1            1859          1867         historical              tas   
2            1868          1876         historical              tas   
3            1877          1885         historical              tas   
4            1886          1894         historical              tas   

  archive_model archive_ensemble       stitching_id archive_start_yr  \
0       CanESM5        r22i1p1f1  ssp245~r1i1p1f1~1             1850   
1       CanESM5        r23i1p1f1  ssp245~r1i1p1f1~1             1868   
2       CanESM5        r25i1p1f1  ssp245~r1i1p1f1~1             1868   
3       CanESM5         r1i1p1f1  ssp245~r1i1p1f1~1             1877   
4       CanESM5        r21i1p1f1  ssp245~r1i1p1f1~1             1868   

  archive_end_yr                     

# new block
- same code
- run after the fixes

In [6]:
my_recipes = stitches.make_recipe(target_data, archive_data,
                                  tol = 0.06, N_matches = 4, reproducible=True)


You have requested more recipes than possible for at least one target trajectories, returning what can


# old-new comparison

In [8]:

old_recipes = pd.read_csv('old_recipes.csv')
print('------------------------------------------------------------------')
print('Are the old and new recipes identical:')
print(my_recipes.compare(old_recipes).empty)
print('------------------------------------------------------------------')
# two data frames identcal if pd.DataFrame.compare returns an empty df,
# since pd.DataFrame.compare returns a data frame of all differences

------------------------------------------------------------------
Are the old and new recipes identical:
True
------------------------------------------------------------------


In [9]:
stitched_global_temp = stitches.gmat_stitching(my_recipes)
old_global_temp = pd.read_csv('old_tgavs.csv')
print('------------------------------------------------------------------')
print('Are the old and new recipes identical:')
print(stitched_global_temp.compare(old_global_temp).empty)
print('------------------------------------------------------------------')
print('max differences of value column:')
print(np.max(abs(old_global_temp.value - stitched_global_temp.value)))
print('------------------------------------------------------------------')
print('Actual disagreements:')
print(stitched_global_temp.compare(old_global_temp))
print('------------------------------------------------------------------')

------------------------------------------------------------------
Are the old and new recipes identical:
False
------------------------------------------------------------------
max differences of value column:
4.440892098500626e-16
------------------------------------------------------------------
Actual disagreements:
        value          
         self     other
229  2.839796  2.839796
491  2.785270  2.785270
751  2.785270  2.785270
------------------------------------------------------------------
